In [1]:
import pandas as pd
import numpy as np

# ==========================
# 1. Load the Dataset
# ==========================
file_path = "/Users/User2/Desktop/Research Seminar/approach_1_df.csv"
# Use low_memory=False to avoid dtype warnings
df = pd.read_csv(file_path, low_memory=False)

# ==========================
# 2. Date Conversion and Feature Extraction
# ==========================
# Define date columns; note: any invalid date (e.g., wrong format) becomes NaT (Not a Time)
date_columns = ["bldat", "budat", "cpudt", "aedat", "augdt"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')  # 'coerce' makes invalid dates into NaT

# Sort by posting date (budat) to maintain time order
df = df.sort_values(by="budat")

# Extract date features from 'budat'
df["year"] = df["budat"].dt.year
df["month"] = df["budat"].dt.month
df["day"] = df["budat"].dt.day
df["day_of_week"] = df["budat"].dt.weekday  # Monday=0, Sunday=6
df["quarter"] = df["budat"].dt.quarter

# Create cyclical (sin/cos) encoding for month and day-of-week to capture periodicity
df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)
df["day_sin"] = np.sin(2 * np.pi * df["day_of_week"] / 7)
df["day_cos"] = np.cos(2 * np.pi * df["day_of_week"] / 7)

# Option: If you no longer need the original date columns, drop them.
df = df.drop(columns=date_columns)

# ==========================
# 3. Missing Value Analysis and Imputation
# ==========================
# Calculate percentage of missing values per column
missing_perc = df.isna().mean() * 100

# Identify columns based on missing value thresholds
high_missing_cols = missing_perc[missing_perc > 90].index.tolist()
moderate_missing_cols = missing_perc[(missing_perc > 20) & (missing_perc <= 90)].index.tolist()
low_missing_cols = missing_perc[missing_perc <= 20].index.tolist()

print("Columns with >90% missing:", high_missing_cols)
print("Columns with 20-90% missing:", moderate_missing_cols)
print("Columns with <=20% missing:", low_missing_cols)

# Identify binary columns: we check numeric columns where the unique non-missing values are only 0 and 1.
binary_cols = []
for col in df.columns:
    # Only check columns that can be treated as numeric (if not, skip this check)
    try:
        unique_vals = df[col].dropna().unique()
        # If the set of unique values is a subset of {0, 1}, then we consider it binary.
        if set(unique_vals).issubset({0, 1}):
            binary_cols.append(col)
    except Exception as e:
        continue

print("Identified binary columns:", binary_cols)

# For binary columns, fill missing values with 0
df[binary_cols] = df[binary_cols].fillna(0)

# For numeric columns that are not binary, impute missing values.
# First, identify numeric columns (as numbers) and remove the binary ones.
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
non_binary_numeric_cols = list(set(numeric_cols) - set(binary_cols))

# Impute non-binary numeric columns with median.
# This is applied both to moderate and high missing rate columns.
for col in non_binary_numeric_cols:
    if df[col].isna().sum() > 0:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

# For categorical (non-numeric) columns, fill missing values with the mode.
categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()
for col in categorical_cols:
    if df[col].isna().sum() > 0:
        mode_val = df[col].mode().iloc[0]
        df[col] = df[col].fillna(mode_val)

# ==========================
# 4. Categorical Encoding
# ==========================
# At this stage, all columns have no missing values.
# We now encode categorical variables to make the entire dataset numeric.
# “Number of features remain consistent” means that every sample ends up with the same number of numeric columns.

# Define a threshold to separate high-cardinality and low-cardinality categorical columns
cardinality_threshold = 20
high_card_cols = [col for col in categorical_cols if df[col].nunique() > cardinality_threshold]
low_card_cols = [col for col in categorical_cols if df[col].nunique() <= cardinality_threshold]

print("High-cardinality categorical columns:", high_card_cols)
print("Low-cardinality categorical columns:", low_card_cols)

# Frequency encoding for high-cardinality categorical columns:
for col in high_card_cols:
    freq_map = df[col].value_counts().to_dict()
    df[col + "_freq"] = df[col].map(freq_map)

# One-hot encoding for low-cardinality categorical columns:
df = pd.get_dummies(df, columns=low_card_cols, drop_first=True)

# ==========================
# 5. Save the Preprocessed Dataset
# ==========================
# Now, the entire dataset is numeric with no missing values and a fixed set of features.
# This dataset is ready for your GAN (without any dimensionality reduction).
output_file = "/Users/User2/Desktop/Research Seminar/processed_timegan_data_full.csv"
df.to_csv(output_file, index=False)
print(f"✅ Preprocessing complete! Processed dataset saved as '{output_file}'.")

Columns with >90% missing: ['bvorg', 'stblg', 'stjah', 'bktxt', 'bstat', 'grpid', 'xsnet', 'ausbk', 'stgrd', 'xreversal', 'augcp', 'augbl', 'umskz', 'umsks', 'txbhw', 'txbfw', 'hwbas', 'fwbas', 'mwart', 'sgtxt', 'vbund', 'kostl', 'vbeln', 'vbel2', 'posn2', 'xhres', 'xskrl', 'xauto', 'xzahl', 'zbd3t', 'zbd2p', 'sknto', 'wskto', 'zlspr', 'nebtr', 'mwsk1', 'dmbt1', 'dmbt2', 'rebzg', 'rebzj', 'rebzz', 'rebzt', 'landl', 'klibt', 'zekkn', 'bualt', 'rdiff', 'rdif2', 'txjcd', 'sknt2', 'hwmet', 'xragl', 'stbuk', 'txbh2', 'pprct', 'xnegp', 'kkber', 'agzei', 'auggj', 'segment', 'kstar']
Columns with 20-90% missing: ['xblnr', 'kursf', 'xrueb', 'kurs2', 'xmwst', 'doccat', 'buzid', 'gsber', 'mwskz', 'txgrp', 'valut', 'zuonr', 'xumsw', 'xkres', 'xopvw', 'xncop', 'saknr', 'kunnr', 'lifnr', 'gvtyp', 'zfbdt', 'zterm', 'zbd1t', 'zbd2t', 'zbd1p', 'skfbt', 'matnr', 'werks', 'menge', 'meins', 'erfmg', 'erfme', 'bpmng', 'bprme', 'ebeln', 'ebelp', 'vprsv', 'bwkey', 'bustw', 'prctr', 'xref3', 'kidno', 'squan']

/var/folders/qm/qt_g3_ls4hx6bc9jsnngjcw80000gq/T/ipykernel_85827/3741710448.py:107: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col + "_freq"] = df[col].map(freq_map)
/var/folders/qm/qt_g3_ls4hx6bc9jsnngjcw80000gq/T/ipykernel_85827/3741710448.py:107: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col + "_freq"] = df[col].map(freq_map)
/var/folders/qm/qt_g3_ls4hx6bc9jsnngjcw80000gq/T/ipykernel_85827/3741710448.py:107: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` 

✅ Preprocessing complete! Processed dataset saved as '/Users/User2/Desktop/Research Seminar/processed_timegan_data_full.csv'.


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# ==========================
# 1. Load the Dataset
# ==========================
file_path = "/Users/User2/Desktop/Research Seminar/approach_1_df.csv"
# Use low_memory=False to avoid dtype warnings
df = pd.read_csv(file_path, low_memory=False)

# ==========================
# 2. Date Conversion and Feature Extraction
# ==========================
# Define date columns; invalid dates (e.g. badly formatted strings) become NaT
date_columns = ["bldat", "budat", "cpudt", "aedat", "augdt"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Sort by posting date ('budat') to maintain time order
df = df.sort_values(by="budat")

# Extract date features from 'budat'
df["year"] = df["budat"].dt.year
df["month"] = df["budat"].dt.month
df["day"] = df["budat"].dt.day
df["day_of_week"] = df["budat"].dt.weekday  # Monday=0, Sunday=6
df["quarter"] = df["budat"].dt.quarter

# Create cyclical (sin/cos) encoding for month and day-of-week
df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)
df["day_sin"] = np.sin(2 * np.pi * df["day_of_week"] / 7)
df["day_cos"] = np.cos(2 * np.pi * df["day_of_week"] / 7)

# Optionally drop the original date columns if you no longer need them
df = df.drop(columns=date_columns)

# ==========================
# 3. Missing Value Analysis and Imputation
# ==========================
# Compute percentage of missing values per column
missing_perc = df.isna().mean() * 100

# Identify columns by missing value thresholds:
high_missing_cols = missing_perc[missing_perc > 90].index.tolist()
moderate_missing_cols = missing_perc[(missing_perc > 20) & (missing_perc <= 90)].index.tolist()
low_missing_cols = missing_perc[missing_perc <= 20].index.tolist()

print("Columns with >90% missing:", high_missing_cols)
print("Columns with 20-90% missing:", moderate_missing_cols)
print("Columns with <=20% missing:", low_missing_cols)

# Identify binary columns by checking if the unique non-missing values are only 0 and 1.
binary_cols = []
for col in df.columns:
    try:
        unique_vals = df[col].dropna().unique()
        if set(unique_vals).issubset({0, 1}):
            binary_cols.append(col)
    except Exception:
        continue
print("Identified binary columns:", binary_cols)

# Impute binary columns with 0 (if there are any missing values)
df[binary_cols] = df[binary_cols].fillna(0)

# For numeric (non-binary) columns, impute missing values with the median.
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
non_binary_numeric_cols = list(set(numeric_cols) - set(binary_cols))
for col in non_binary_numeric_cols:
    if df[col].isna().sum() > 0:
        df[col] = df[col].fillna(df[col].median())

# For categorical (non-numeric) columns, impute missing values with the mode.
categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()
for col in categorical_cols:
    if df[col].isna().sum() > 0:
        df[col] = df[col].fillna(df[col].mode().iloc[0])

# ==========================
# 4. Categorical Encoding
# ==========================
# Define a cardinality threshold to separate high- and low-cardinality columns
cardinality_threshold = 20
high_card_cols = [col for col in categorical_cols if df[col].nunique() > cardinality_threshold]
low_card_cols = [col for col in categorical_cols if df[col].nunique() <= cardinality_threshold]

print("High-cardinality categorical columns:", high_card_cols)
print("Low-cardinality categorical columns:", low_card_cols)

# Frequency encoding for high-cardinality categorical columns
# (Mapping each category to its frequency count)
for col in high_card_cols:
    freq_map = df[col].value_counts().to_dict()
    df[col + "_freq"] = df[col].map(freq_map)

# One-hot encoding for low-cardinality categorical columns
df = pd.get_dummies(df, columns=low_card_cols, drop_first=True)

# At this point, the entire dataset is numeric with no NaNs.
# Save the preprocessed data before scaling (optional)
preproc_output = "/Users/User2/Desktop/Research Seminar/processed_timegan_data_full.csv"
df.to_csv(preproc_output, index=False)
print(f"✅ Preprocessing complete! Processed dataset saved as '{preproc_output}'.")

Columns with >90% missing: ['bvorg', 'stblg', 'stjah', 'bktxt', 'bstat', 'grpid', 'xsnet', 'ausbk', 'stgrd', 'xreversal', 'augcp', 'augbl', 'umskz', 'umsks', 'txbhw', 'txbfw', 'hwbas', 'fwbas', 'mwart', 'sgtxt', 'vbund', 'kostl', 'vbeln', 'vbel2', 'posn2', 'xhres', 'xskrl', 'xauto', 'xzahl', 'zbd3t', 'zbd2p', 'sknto', 'wskto', 'zlspr', 'nebtr', 'mwsk1', 'dmbt1', 'dmbt2', 'rebzg', 'rebzj', 'rebzz', 'rebzt', 'landl', 'klibt', 'zekkn', 'bualt', 'rdiff', 'rdif2', 'txjcd', 'sknt2', 'hwmet', 'xragl', 'stbuk', 'txbh2', 'pprct', 'xnegp', 'kkber', 'agzei', 'auggj', 'segment', 'kstar']
Columns with 20-90% missing: ['xblnr', 'kursf', 'xrueb', 'kurs2', 'xmwst', 'doccat', 'buzid', 'gsber', 'mwskz', 'txgrp', 'valut', 'zuonr', 'xumsw', 'xkres', 'xopvw', 'xncop', 'saknr', 'kunnr', 'lifnr', 'gvtyp', 'zfbdt', 'zterm', 'zbd1t', 'zbd2t', 'zbd1p', 'skfbt', 'matnr', 'werks', 'menge', 'meins', 'erfmg', 'erfme', 'bpmng', 'bprme', 'ebeln', 'ebelp', 'vprsv', 'bwkey', 'bustw', 'prctr', 'xref3', 'kidno', 'squan']

/var/folders/qm/qt_g3_ls4hx6bc9jsnngjcw80000gq/T/ipykernel_85827/2829845349.py:96: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col + "_freq"] = df[col].map(freq_map)
/var/folders/qm/qt_g3_ls4hx6bc9jsnngjcw80000gq/T/ipykernel_85827/2829845349.py:96: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col + "_freq"] = df[col].map(freq_map)
/var/folders/qm/qt_g3_ls4hx6bc9jsnngjcw80000gq/T/ipykernel_85827/2829845349.py:96: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` man

✅ Preprocessing complete! Processed dataset saved as '/Users/User2/Desktop/Research Seminar/processed_timegan_data_full.csv'.


In [4]:
# ==========================
# 5. Scaling: MinMax Scaler for Non-Binary Columns
# ==========================
# Identify non-binary columns (all columns except those identified as binary)
all_columns = df.columns.tolist()
non_binary_cols = [col for col in all_columns if col not in binary_cols]

# Fill missing values for binary columns only if they exist in the DataFrame.
existing_binary_cols = [col for col in binary_cols if col in df.columns]
df[existing_binary_cols] = df[existing_binary_cols].fillna(0)

# Apply MinMax scaling to non-binary columns only.
scaler = MinMaxScaler()
df[non_binary_cols] = scaler.fit_transform(df[non_binary_cols])

# Save the final scaled dataset
scaled_output = "/Users/User2/Desktop/Research Seminar/timegan_minmax_scaled_data.csv"
df.to_csv(scaled_output, index=False)
print(f"✅ MinMax scaling applied to non-binary columns. Final dataset saved as '{scaled_output}'.")

ValueError: could not convert string to float: '10:06:02'

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# ==========================
# 1. Load Preprocessed Dataset
# ==========================
preprocessed_file = "/Users/User2/Desktop/Research Seminar/processed_timegan_data_full.csv"
df = pd.read_csv(preprocessed_file, low_memory=False)

print("✅ Loaded preprocessed dataset with shape:", df.shape)
print("✅ Columns:", df.columns.tolist())

# ==========================
# 2. Convert Time-Like String Columns to Numeric (Seconds)
# ==========================
# If you know which columns contain time-like strings (e.g., "HH:MM:SS"),
# list them here. For example, 'cputm' might have values like "10:06:02".
time_like_cols = ["cputm"]  # <-- Adjust this list for your actual time-based columns

for col in time_like_cols:
    if col in df.columns:
        # Convert string "HH:MM:SS" to pandas datetime (coerce invalid formats to NaT)
        df[col] = pd.to_datetime(df[col], format="%H:%M:%S", errors="coerce")

        # If you prefer, fill NaT (missing after conversion) with 0:0:0
        df[col] = df[col].fillna(pd.to_datetime("00:00:00"))

        # Convert to total seconds: hour*3600 + minute*60 + second
        df[col] = df[col].dt.hour * 3600 + df[col].dt.minute * 60 + df[col].dt.second

# ==========================
# 3. Identify Binary Columns
# ==========================
# We'll check numeric columns whose unique non-missing values are a subset of {0, 1}.
binary_cols = []
for col in df.columns:
    # Only consider columns that are numeric; skip if object/string
    if pd.api.types.is_numeric_dtype(df[col]):
        unique_vals = set(df[col].dropna().unique())
        if unique_vals.issubset({0, 1}):
            binary_cols.append(col)

print("✅ Identified binary columns:", binary_cols)

# ==========================
# 4. Select Numeric, Non-Binary Columns for MinMax Scaling
# ==========================
non_binary_numeric_cols = []
for col in df.columns:
    # We only scale columns that are numeric and not in binary_cols
    if pd.api.types.is_numeric_dtype(df[col]) and col not in binary_cols:
        non_binary_numeric_cols.append(col)

print("✅ Non-binary numeric columns (will be scaled):", non_binary_numeric_cols)

# ==========================
# 5. Apply MinMax Scaler to Non-Binary Numeric Columns
# ==========================
scaler = MinMaxScaler()
df[non_binary_numeric_cols] = scaler.fit_transform(df[non_binary_numeric_cols])

# ==========================
# 6. Save the Fully Scaled Dataset
# ==========================
scaled_output = "/Users/User2/Desktop/Research Seminar/timegan_fully_scaled_data.csv"
df.to_csv(scaled_output, index=False)
print(f"✅ All done! Final scaled dataset saved at '{scaled_output}'.")

✅ Loaded preprocessed dataset with shape: (334626, 285)
✅ Columns: ['mandt', 'belnr', 'gjahr', 'monat', 'cputm', 'wwert', 'usnam', 'xblnr', 'stblg', 'stjah', 'kursf', 'awkey', 'kurs2', 'basw2', 'umrd2', 'curt2', 'stgrd', 'xreversal', 'recordstamp_x', 'buzei', 'augcp', 'augbl', 'bschl', 'dmbtr', 'wrbtr', 'pswbt', 'txbhw', 'txbfw', 'hwbas', 'fwbas', 'txgrp', 'valut', 'zuonr', 'vbeln', 'vbel2', 'posn2', 'saknr', 'hkont', 'kunnr', 'lifnr', 'zfbdt', 'zterm', 'zbd1t', 'zbd2t', 'zbd3t', 'zbd1p', 'zbd2p', 'skfbt', 'sknto', 'wskto', 'nebtr', 'dmbt1', 'dmbt2', 'rebzg', 'rebzj', 'rebzz', 'klibt', 'matnr', 'werks', 'menge', 'erfmg', 'bpmng', 'ebeln', 'ebelp', 'zekkn', 'bwkey', 'bualt', 'rdiff', 'rdif2', 'prctr', 'dmbe2', 'sknt2', 'txbh2', 'pprct', 'xref3', 'kidno', 'agzei', 'auggj', 'kstar', 'recordstamp_y', 'year', 'month', 'day', 'day_of_week', 'quarter', 'month_sin', 'month_cos', 'day_sin', 'day_cos', 'cputm_freq', 'wwert_freq', 'usnam_freq', 'xblnr_freq', 'awkey_freq', 'recordstamp_x_freq', 'a

In [5]:
pip install tensorflow==2.9.1  # or later 2.x version

ERROR: Could not find a version that satisfies the requirement tensorflow==2.9.1 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2)
ERROR: No matching distribution found for tensorflow==2.9.1
Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

2025-03-06 02:00:30.920471: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# ==========================
# Load the Scaled Dataset
# ==========================
data_file = "/Users/User2/Desktop/Research Seminar/timegan_fully_scaled_data.csv"
df = pd.read_csv(data_file, low_memory=False)
print("Data shape (N, D):", df.shape)

# Convert to numpy
data = df.values  # shape: (N, D)

# ==========================
# Define Sequence Length
# ==========================
seq_len = 30  # adjust as appropriate for your domain
dim = data.shape[1]  # number of features (D)

# Create overlapping windows of length seq_len
sequences = []
for i in range(len(data) - seq_len + 1):
    seq_slice = data[i : i + seq_len, :]
    sequences.append(seq_slice)

sequences = np.array(sequences, dtype=np.float32)  # shape: (N - seq_len + 1, seq_len, D)
print("Sequences shape:", sequences.shape)

# Optionally, split into training and testing for final evaluation
train_data, test_data = train_test_split(sequences, test_size=0.2, random_state=42)
print("Train shape:", train_data.shape, "| Test shape:", test_data.shape)

Data shape (N, D): (334626, 285)


ValueError: could not convert string to float: '2018-01-25'

In [8]:
print(df.dtypes)
non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns
print("Non-numeric columns:", non_numeric_cols)

mandt                 float64
belnr                 float64
gjahr                 float64
monat                 float64
cputm                 float64
                       ...   
kkber_USA1               bool
squan_-                  bool
squan_0                  bool
operation_flag_y_L       bool
operation_flag_y_U       bool
Length: 285, dtype: object
Non-numeric columns: Index(['wwert', 'usnam', 'xblnr', 'awkey', 'recordstamp_x', 'augcp', 'valut',
       'zuonr', 'kunnr', 'lifnr',
       ...
       'txjcd_CANS', 'txjcd_CAON', 'txjcd_GA0000000', 'hwmet_E', 'kkber_EU01',
       'kkber_USA1', 'squan_-', 'squan_0', 'operation_flag_y_L',
       'operation_flag_y_U'],
      dtype='object', length=195)


In [9]:
for col in non_numeric_cols:
    print(f"Unique values in {col}:", df[col].unique()[:10])

Unique values in wwert: ['2018-01-25' '2018-04-04' '2018-04-06' '2018-04-07' '2018-04-08'
 '2018-04-09' '2018-04-10' '2018-04-11' '2018-04-12' '2018-04-13']
Unique values in usnam: ['SHOBHITSAURA' 'SIBASISM' 'MOUNIKAPALI' 'BHUSHAN' 'GAIKWADPAWAR'
 'VISHAKHA' 'RAJARSHIG' 'PKADARI' 'ANANTHARAMUL' 'MALVIYAH']
Unique values in xblnr: ['TEST' '0080000079' '0080000080' '0080000081' '0080000075' '0080000074'
 '0080000076' '0080000077' '0080000078' '0080000000']
Unique values in awkey: ['1900000000USA12018' '50000325312018' '50000326792018' '50000325372018'
 '50000329072018' '50000329622018' '51056450872018' '51056304442018'
 '51056304452018' '50000477702018']
Unique values in recordstamp_x: ['2022-01-31 20:41:45.693184+00:00' '2022-03-22 05:40:44.237359+00:00'
 '2022-03-22 05:41:07.064797+00:00' '2022-03-22 05:40:45.067314+00:00'
 '2022-03-22 05:41:40.700059+00:00' '2022-03-22 05:41:50.078926+00:00'
 '2022-03-26 02:44:48.288650+00:00' '2022-03-23 08:25:31.925815+00:00'
 '2022-03-22 10:15:32.5

Unique values in bustw_RE14: [False  True]
Unique values in bustw_RE21: [False  True]
Unique values in bustw_WA01: [False  True]
Unique values in bustw_WA04: [False  True]
Unique values in bustw_WE01: [ True False]
Unique values in txjcd_CA0000000: [ True False]
Unique values in txjcd_CANS: [False  True]
Unique values in txjcd_CAON: [False  True]
Unique values in txjcd_GA0000000: [False  True]
Unique values in hwmet_E: [ True False]
Unique values in kkber_EU01: [False  True]
Unique values in kkber_USA1: [False  True]
Unique values in squan_-: [ True False]
Unique values in squan_0: [False  True]
Unique values in operation_flag_y_L: [ True False]
Unique values in operation_flag_y_U: [False  True]


In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# ==========================
# 1) Load the Dataset
# ==========================
file_path = "/Users/User2/Desktop/Research Seminar/timegan_fully_scaled_data.csv"
df = pd.read_csv(file_path, low_memory=False)
print("Original Data Shape:", df.shape)

# --------------------------
# Quick check: which columns are still object/string?
# --------------------------
obj_cols = df.select_dtypes(include=["object"]).columns.tolist()
print("String/object columns:", obj_cols)

# ==========================
# 2) Convert Boolean Columns to 0/1 (if any)
# ==========================
bool_cols = df.select_dtypes(include=["bool"]).columns.tolist()  # or manually identify if needed
if bool_cols:
    print("Converting boolean columns to int:", bool_cols)
    df[bool_cols] = df[bool_cols].astype(int)

# Re-check object columns after bool conversion
obj_cols = df.select_dtypes(include=["object"]).columns.tolist()

# ==========================
# 3) Parse Date/Time-Like Columns
# ==========================
# Some columns clearly have date strings (e.g., '2018-01-25'), others might have date+time stamps.
# We'll parse them as datetime and convert to UNIX seconds. 
# Adjust the list below as needed.

date_like_cols = [
    "wwert",         # had values like '2018-01-25'
    "augcp",         # date-like e.g. '2022-03-28'
    "valut",         # date-like
    "zfbdt",         # date-like
    "recordstamp_x", # has timestamps like '2022-01-31 20:41:45.693184+00:00'
    "recordstamp_y"  # similar
    # etc. Add any other columns that are obviously date/time
]

for col in date_like_cols:
    if col in df.columns:
        print(f"Parsing date/time for column: {col}")
        # Parse date/time
        df[col] = pd.to_datetime(df[col], errors="coerce")
        # Optionally fill NaT with a default date if needed
        df[col] = df[col].fillna(pd.to_datetime("1970-01-01"))
        # Convert to integer seconds (UNIX epoch)
        df[col] = df[col].astype(np.int64) // 10**9

# Re-check object columns after date/time conversion
obj_cols = df.select_dtypes(include=["object"]).columns.tolist()
print("Remaining object columns after parsing date/time:", obj_cols)

Original Data Shape: (334626, 285)
String/object columns: ['wwert', 'usnam', 'xblnr', 'awkey', 'recordstamp_x', 'augcp', 'valut', 'zuonr', 'kunnr', 'lifnr', 'zfbdt', 'matnr', 'prctr', 'pprct', 'recordstamp_y']
Converting boolean columns to int: ['bukrs_C002', 'bukrs_C003', 'bukrs_C004', 'bukrs_C005', 'bukrs_EU01', 'bukrs_USA1', 'blart_DA', 'blart_DR', 'blart_DZ', 'blart_KG', 'blart_KR', 'blart_KZ', 'blart_PR', 'blart_RE', 'blart_RV', 'blart_SA', 'blart_WA', 'blart_WE', 'blart_WL', 'tcode_FB08', 'tcode_FB1D', 'tcode_FB60', 'tcode_FB70', 'tcode_FBM1', 'tcode_FBZ1', 'tcode_FBZ2', 'tcode_MB01', 'tcode_MB1C', 'tcode_MIGO_GI', 'tcode_MIGO_GR', 'tcode_MIRO', 'tcode_MR21', 'tcode_MR8M', 'tcode_VF01', 'tcode_VF02', 'tcode_VF11', 'tcode_VL02N', 'tcode_VL09', 'bvorg_1900000001C00222', 'bvorg_1900000002C00222', 'bktxt_Initial stock upload', 'waers_CNY', 'waers_EUR', 'waers_JPY', 'waers_USD', 'glvor_RFIG', 'glvor_RMPR', 'glvor_RMRP', 'glvor_RMWA', 'glvor_RMWE', 'glvor_RMWL', 'glvor_SD00', 'grpid_IV

In [38]:
# ==========================
# 4) Frequency Encode Remaining String Columns
# ==========================
# For columns like 'usnam', 'xblnr', 'awkey', 'kunnr', 'lifnr', 'matnr', 'prctr', 'pprct', 'zuonr' etc.
# We'll do a quick frequency encoding. 
# If a column is actually numeric but stored as string, you may parse it, but let's keep it simple.

for col in obj_cols:
    print(f"Frequency encoding column: {col}")
    freq_map = df[col].value_counts().to_dict()
    df[col] = df[col].map(freq_map)

# Now all object columns should be numeric
obj_cols_after = df.select_dtypes(include=["object"]).columns.tolist()
if obj_cols_after:
    print("Warning: some columns are still object type:", obj_cols_after)
else:
    print("✅ All columns are now numeric.")

# ==========================
# 5) Verify
# ==========================
print(df.info())
print("Sample of DataFrame:\n", df.head(3))


from sklearn.preprocessing import MinMaxScaler

# --------------------------
# 5.5) Scale Date/Time and Frequency-Encoded Columns
# --------------------------
# Define the columns that you want to scale:
date_cols_to_scale = ["wwert", "augcp", "valut", "zfbdt", "recordstamp_x", "recordstamp_y"]
freq_cols = ["usnam", "xblnr", "awkey", "zuonr", "kunnr", "lifnr", "matnr", "prctr", "pprct"]

# Combine the lists (and ensure they exist in df)
cols_to_scale = [col for col in (date_cols_to_scale + freq_cols) if col in df.columns]

print("Scaling the following columns:", cols_to_scale)

# Initialize the scaler and fit_transform on these columns
scaler = MinMaxScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

# Optional: Print summary statistics to verify they are now between 0 and 1
print(df[cols_to_scale].describe())


# ==========================
# 6) Reshape into (N - seq_len + 1, seq_len, D)
# ==========================
data = df.values  # shape (N, D)
seq_len = 30      # or whichever
N, D = data.shape
print(f"Data shape (N, D): {(N, D)}")

# Create sequences
sequences = []
for i in range(N - seq_len + 1):
    seq_slice = data[i : i + seq_len, :]
    sequences.append(seq_slice)

sequences = np.array(sequences, dtype=np.float32)  # shape: (N - seq_len + 1, seq_len, D)
print("Sequences shape:", sequences.shape)

# Optional train/test split
train_data, test_data = train_test_split(sequences, test_size=0.2, random_state=42)
print("Train shape:", train_data.shape, "| Test shape:", test_data.shape)

# ==========================
# 7) Proceed to TimeGAN Model
# ==========================
# (At this point, everything is numeric, so you can feed 'train_data' into the TimeGAN pipeline.)

Frequency encoding column: usnam
Frequency encoding column: xblnr
Frequency encoding column: awkey
Frequency encoding column: zuonr
Frequency encoding column: kunnr
Frequency encoding column: lifnr
Frequency encoding column: matnr
Frequency encoding column: prctr
Frequency encoding column: pprct
✅ All columns are now numeric.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334626 entries, 0 to 334625
Columns: 285 entries, mandt to operation_flag_y_U
dtypes: float64(90), int64(195)
memory usage: 727.6 MB
None
Sample of DataFrame:
    mandt     belnr  gjahr     monat     cputm       wwert  usnam   xblnr  \
0    1.0  0.198683    0.0  0.000000  0.420839  1516838400    890  254627   
1    1.0  0.198683    0.0  0.000000  0.420839  1516838400    890  254627   
2    1.0  0.538970    0.0  0.272727  0.236498  1522800000   4566  254627   

      stblg  stjah  ...  txjcd_CANS  txjcd_CAON  txjcd_GA0000000  hwmet_E  \
0  0.006623    0.0  ...           0           0                0        1   
1  

In [61]:
non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()
print("Non-numeric columns remaining:", non_numeric_cols)

Non-numeric columns remaining: []


In [60]:
# ==========================
# Export Processed DataFrame
# ==========================
output_path = "/Users/User2/Desktop/Research Seminar/timegan_processed_data.csv"
df.to_csv(output_path, index=False)
print(f"Processed data saved to {output_path}")

Processed data saved to /Users/User2/Desktop/Research Seminar/timegan_processed_data.csv
